# Text categorization
The goal of this program is to be able to categorize text into categories in the Brown corpus.
Text categorization as a whole can be important and useful in many ways. Being able to easily categorize big amounts of documents can for example help with research. Automatic categorization can also make texts more easily available to people who are looking for certain types of texts.


First we import some modules:

In [1]:
# If you want to plot graphics in your notebook, keep this %matplotlib command here before your imports
%matplotlib notebook

from collections import Counter
import nltk
import numpy
from nltk.util import ngrams
import random
import math    
from nltk.corpus import brown


### Features
Next define some functions that are needed for the program to work. These functions find certain features that exist in the data set and make a document more likely to belong to a certain category.

The next function finds which words a certain document contains

In [2]:
def contains_features(document,word_features):
    document_words = set(document) 
    features = {}
    label = "contains"
    for word in word_features:
        features['{:s}({:s})'.format(label,word)] = (word in document_words)
    return features

The next function finds the thirty first words of the document

In [3]:
def first_words(document,word_features):
    document = document[:30]
    document_words = set(document)
    first_words = {}
    label = "begins"
    for word in word_features:
        first_words['{:s}({:s})'.format(label,word)] = (word in document_words)
    return first_words

The next function finds the thirty last words of the document

In [4]:
def last_words(document,word_features):
    document = document[-30:]
    document_words = set(document)
    last_words = {}
    label = "ends"
    for word in word_features:
        last_words['{:s}({:s})'.format(label,word)] = (word in document_words)
    return last_words

The next function finds which bigrams the document contains

In [6]:
def get_bigram_features(document,bigrams_most_common):
    document = ["{:s} {:s}".format(w1, w2) for w1, w2 in nltk.bigrams(document)]
    bigrams = {}
    label = "bigram"
    for bigram in bigrams_most_common:
        bigrams['{:s}({:s})'.format(label,bigram)] = (bigram in document)
    return bigrams

The following function finds trigrams in a document

In [7]:
def get_trigram_features(document,trigrams_most_common):
    document = ["{:s} {:s} {:s}".format(w1, w2, w3) for w1, w2, w3 in nltk.trigrams(document)]
    trigrams = {}
    label = "trigram"
    for trigram in trigrams_most_common:
        trigrams['{:s}({:s})'.format(label,trigram)] = (trigram in document)
    return trigrams

The following function combines all the other functions

In [8]:
def get_all_features_combined(document,word_features,bigrams_most_common,trigrams_most_common):
    return {**contains_features(document, word_features),
            **first_words(document, word_features),
            **last_words(document, word_features),
            **get_bigram_features(document,bigrams_most_common),
            **get_trigram_features(document,trigrams_most_common)}

### Splitting the data
In the next cell we define the used data set and split it into a training set, a validation set and a test set. The training and validation sets are random and change every time the cell is run. These two sets are random because the used documents are shuffled in the beginning after the test set has been extracted from the whole data set. The documents are split into the training and the validation sets at the end of the next cell after the documents have been shuffled and all the document features have been found. The training set is 90% of the whole data and the test and validation sets are both 5% of all the documents. 

The test set is every 20th document from the end of the whole list of documents. This is because the documents in the list "documents" are ordered by category. By starting from the end of the list we are able to get all, even the smaller categories like science fiction, into the test set. If took every 20th document from the beginning the test set would not contain any science fiction texts.

If you want to train the classifier again you have to run the next cell before running the cell where the training happens. The cell is slowish to run because the amount of both bigrams and trigrams is 3000 to make the classifier perform a little better.

In [9]:
documents = [(list(brown.words(fileid)), category)
            for category in brown.categories()
            for fileid in brown.fileids(category)]

#Here we get every 20th document of all the documents and make them the test set.
test_set = documents[::-20]

#Here we remove every item that is in the test set from the list "documents" so that the test set documents don't 
#appear in the training or validation sets.
for item in documents:
    if item in test_set:
        documents.remove(item)

#Here we shuffle the documents to make the training and validation sets random.
random.shuffle(documents)

all_words = nltk.FreqDist(w.lower() for w in brown.words())
#We use only the 3000 most common words so that the program doesn't get too slow
word_features = [w for w, f in all_words.most_common(3000)] 
    
all_bigrams = nltk.FreqDist((w1.lower(), w2.lower()) for w1, w2 in nltk.bigrams(brown.words()))
all_trigrams = nltk.FreqDist((w1.lower(), w2.lower(), w3.lower()) for w1, w2, w3 in nltk.trigrams(brown.words()))

#Here we get the 3000 most commmon bigrams and trigrams out of all the bigrams in the corpus
bigrams_most_common = ["{:s} {:s}".format(w1, w2) for ((w1, w2), f) in all_bigrams.most_common(3000)]
trigrams_most_common = ["{:s} {:s} {:s}".format(w1, w2, w3) for ((w1, w2, w3), f) in all_trigrams.most_common(3000)]

#Now we get all the features for all the documents
featuresets = [(get_all_features_combined(d,word_features,bigrams_most_common,trigrams_most_common), c) for (d,c) in documents]
test_set = [(get_all_features_combined(d,word_features,bigrams_most_common,trigrams_most_common), c) for (d,c) in test_set]

#Here we split the featuresets into a train set and a test set
train_set, validation_set = featuresets[25:], featuresets[:25]



### Classifiers
We use two algorithms to classify the texts: Naive Bayes and decision tree. This way we can compare the results and the classifiers' perfomances.

### Naive Bayes
A Naive Bayes classifier is a supervised machine learning algorithm, i.e. it is given the correct classifications for the train and test set. When the program is trained on the training set it finds features that make the text more likely to be in a certain category. Then when the program is run on the test set it looks which features the documents in the test set contain and calculates the propability of a document belonging to a certain category.

The Naive Bayes algorithm assumes independence of the features in the data. That means that it assumes that the different features are not dependent on each other and that a certain feature doesn't make another feature more or less likely.


In [10]:
#The classifier is trained on the train set
naive_bayes_classifier = nltk.NaiveBayesClassifier.train(train_set)

### Decision tree
Decision tree is an algorithm that can be visualized as a flowchart. "Decision nodes" are nodes that check whether the document contains a certain feature. After going through several "decision nodes" the algorithm gets to a "leaf node" that assigns the category of the document. Decision tree is also a supervised machine learning.

The next cell can be quite slow to run because a decision tree classifier is slower to train.

In [11]:
#The classifier is trained on the train set
decision_tree_classifier = nltk.DecisionTreeClassifier.train(train_set)

### Evaluation during testing
The accuracies of the classifiers are evaluated here by using an already existing command `nltk.classify.accuracy()`. When the command is run on it first categorizes the documents in the given data set and then compares the categorization to the documents' actual categories that can be found in the data set. The evaluation should be run on the validation set because otherwise the classifiers accuracy will seem higher than it actually is. The validation set is a randomized set that is used for testing while developing the program. The actual test set is not random and it is used after the results on the validation set are good enough. This helps us avoid overfitting.

### Majority baseline
Because the most common category is "learned", the majority baseline is how accurate a classifier that classifies all documents as "learned" is. The whole data set consists of 500 documents of which 80 are in the "learned" category. So if we categorized all documents as "learned" 80 of them would be correct. 80 is 16% of 500 so it would mean that the majority baseline is 16%. To be useful at all the classifiers' accuracies should be more than 16%. 

The majority baseline is 16% also for the test set, because it contains 25 documents, 4 of which are in the category "learned". If we label all documents in the set as "learner", 4 documents will be classified correctly and 4 is 16% of 25.

In [12]:
#Here we get all the categories and how many documents each category contains in both the whole data set and
#only the test set.
print("Categories in the whole data:")
categories = list(category for category in brown.categories() for fileid in brown.fileids(category))
print(Counter(categories))
print("Categories in the test set:")
categories = list(category for category in brown.categories() for fileid in brown.fileids(category))[::-20]
print(Counter(categories))

Categories in the whole data:
Counter({'learned': 80, 'belles_lettres': 75, 'lore': 48, 'news': 44, 'hobbies': 36, 'government': 30, 'adventure': 29, 'fiction': 29, 'romance': 29, 'editorial': 27, 'mystery': 24, 'religion': 17, 'reviews': 17, 'humor': 9, 'science_fiction': 6})
Categories in the test set:
Counter({'learned': 4, 'belles_lettres': 4, 'lore': 3, 'news': 2, 'hobbies': 2, 'fiction': 2, 'science_fiction': 1, 'romance': 1, 'reviews': 1, 'religion': 1, 'mystery': 1, 'government': 1, 'editorial': 1, 'adventure': 1})


In the next cell we evaluate the classifiers on the validation set. This is not the final result and not the "actual" accuracy of the classifiers. The accuracy that really matters is the one that we get on the test set. We will get that accuracy later.

In [13]:
print('Accuracy of the Naive Bayes classifier: ')
#We get the accuracy of the classifier by running it on the validation set
print(nltk.classify.accuracy(naive_bayes_classifier, validation_set))
print('Accuracy of the decision tree classifier')
print(nltk.classify.accuracy(decision_tree_classifier, validation_set))

Accuracy of the Naive Bayes classifier: 
0.56
Accuracy of the decision tree classifier
0.24


### Most informative features
In the next cell we print 30 features that the classifier has calculated to have the biggest effect on which category a document belongs to.

In [14]:
#We print 50 most informative features in the data.
naive_bayes_classifier.show_most_informative_features(50)

Most Informative Features
        contains(didn't) = True           myster : learne =     45.5 : 1.0
            bigram(me ,) = True            humor : learne =     35.1 : 1.0
          contains(dark) = True           advent : learne =     33.9 : 1.0
        contains(season) = True           review : learne =     32.9 : 1.0
        contains(wasn't) = True           myster : learne =     32.8 : 1.0
      contains(wouldn't) = True           myster : learne =     32.8 : 1.0
          bigram(you '') = True           romanc : learne =     30.9 : 1.0
        bigram(his wife) = True            humor : learne =     29.7 : 1.0
         bigram(and she) = True           romanc : learne =     29.0 : 1.0
           contains(ran) = True           advent : learne =     28.7 : 1.0
        contains(walked) = True           advent : learne =     28.7 : 1.0
         bigram(asked .) = True           scienc : learne =     28.4 : 1.0
        bigram(his mind) = True           scienc : learne =     28.4 : 1.0

As you can see, most of the most informative are the bigrams, trigrams and words that a document contains. The other features do not show up among the most informative features that often.

Let's see what the list of most informative features means. Let's imagine we have the following row in the list:

contains(didn't) = True    myster:learne =   44.9  :  1.0

The first "column" tells us which feature we are talking about, in this case it's the words a document contains and the word is "didn't". The "True" after the equal sign tells us that the document should contain this feature. If the word after the equal sign was "False" it would mean that the feature should not be in the document. The next column shows which two categories we are comparing and the numbers after the categories tell the likelihood of the cateogries when the document contains the feature. In this case this means that if the document contains the word "didn't" it is 44.9% more likely to be in the category "mystery" than in the "learned" category.

It seems that the second category of the two compared categories is always either "learned" or "belles letres". These are the two biggest categories in the data set. That could mean that because they are so common it is easier for the classifier to find features that would not make a document belong to these categories than to find features that make a document to be "learned" or "belles letres".


### Pseudocode of the decision tree classifier

There is no way to get the most informative features for the decision tree classifier as there is for the Naive Bayes classifier. The way to see the rules by which the decision tree classifier categorizes the documents is to print the pseudocode for the classifier. You can change the depth, i.e. how many of "rules" the classifier has created you want to see.


In [15]:
print(decision_tree_classifier.pseudocode(depth=6))

if bigram(, he) == False: 
  if contains(you) == False: 
    if contains(1961) == False: 
      if contains(approval) == False: 
        if contains(woman) == False: 
          if contains(proud) == False: return 'learned'
          if contains(proud) == True: return 'belles_lettres'
        if contains(woman) == True: 
          if contains(3) == False: return 'lore'
          if contains(3) == True: return 'reviews'
      if contains(approval) == True: 
        if contains(3) == False: return 'lore'
        if contains(3) == True: return 'government'
    if contains(1961) == True: 
      if contains(committee) == False: 
        if contains(show) == False: 
          if contains(nation's) == False: return 'government'
          if contains(nation's) == True: return 'belles_lettres'
        if contains(show) == True: 
          if contains(nation's) == False: return 'learned'
          if contains(nation's) == True: return 'editorial'
      if contains(committee) == True: 
        if 

The pseudocode is not as easy to read as the most informative features of the Naive Bayes classifier, but it can still be quite informative. For example, if we have the following pseudocode:

if bigram(, he) == False: 

&nbsp;&nbsp;&nbsp;&nbsp;  if contains(you) == False:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    if contains(projects) == False:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;      if bigram(when it) == False:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        if bigram(a part) == False: 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;          if contains(sign) == False: return 'learned'

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;          if contains(sign) == True: return 'news'
          
Here we see that if the document doesn't contain the bigram ", he" and it doesn't contain the word "you" and it doesn't contain that word "projects" and it doesn't contain the bigram "when it" and it doesn't contain the bigram "a part" and it doesn't contain the word "sign", the document is classified as "learned. If the document doesn't contain all the other words but does contain the word "sign", it is classified as "news".

Even though the pseudocode does show us the rules the classifier uses classifies the documents, they don't make a lot of sense. It is very hard to know why the bigram ", he" would make a document less likely to belong to a certain category.

## Final evaluation and categorization examples

Here we see the actual accuracy when the program is run on the test set that is always the same and does not depend on the run. We also see how the classifiers classify the documents in the test set.

In [16]:
print('Actual accuracy of the Naive Bayes classifier: ')
#We get the real accuracy of the classifier by running it on the test set
print(nltk.classify.accuracy(naive_bayes_classifier, test_set))
print()

print('Actual accuracy of the decision tree classifier: ')
#We get the real accuracy of the classifier by running it on the test set
print(nltk.classify.accuracy(decision_tree_classifier, test_set))
print()

#Here we get the fileids in the same order as when we define the list "documents". This way we can get the file ids 
#for the documents in the test set.
fileids = [fileid for category in brown.categories()
            for fileid in brown.fileids(category)]
fileids = fileids[::-20]

#Now we print the beginnings of the documents in the test set, how they are categorized by the program and their real 
#categories.
for id in fileids:
    print(brown.words(id))
    print("Category given by the Naive Bayes classifier:")
    print(naive_bayes_classifier.classify(get_all_features_combined(list(brown.words(id)), word_features, bigrams_most_common, trigrams_most_common)))
    print("Category given by the decision tree classifier:")
    print(decision_tree_classifier.classify(get_all_features_combined(list(brown.words(id)), word_features, bigrams_most_common, trigrams_most_common)))
    print("Real category:")
    print(brown.categories(id))
    print()             

Actual accuracy of the Naive Bayes classifier: 
0.64

Actual accuracy of the decision tree classifier: 
0.28

['It', 'would', 'have', 'killed', 'you', 'in', 'the', ...]
Category given by the Naive Bayes classifier:
fiction
Category given by the decision tree classifier:
religion
Real category:
['science_fiction']

['``', 'They', 'make', 'us', 'conformists', 'look', ...]
Category given by the Naive Bayes classifier:
romance
Category given by the decision tree classifier:
fiction
Real category:
['romance']

['Radio', 'is', 'easily', 'outdistancing', ...]
Category given by the Naive Bayes classifier:
belles_lettres
Category given by the decision tree classifier:
religion
Real category:
['reviews']

['Few', 'persons', 'who', 'join', 'the', 'Church', ...]
Category given by the Naive Bayes classifier:
learned
Category given by the decision tree classifier:
news
Real category:
['religion']

['At', 'last', 'the', 'White', 'House', 'is', 'going', ...]
Category given by the Naive Bayes classifie

### Confusion matrixes

Next we make a confusion matrixes for the classifiers. It is quite easy to compare how well the classifiers have categorized the documents in the test set. 

The rows in the matrix are the reference or the gold standard and the columns show the test performance. The numbers inside "<>" is the percentage of documents in the category classified correctly.

In [17]:
#Gold is the real category of each document in the test set.
gold = []
for item in test_set:
    gold.append(item[1])
    
#The confusion tree for the Naive Bayes classifier
test = naive_bayes_classifier.classify_many(f for (f, c) in test_set)
cm = nltk.ConfusionMatrix(gold, test)
print(cm.pretty_format(sort_by_count=False, show_percents=True, truncate=15))

                |                                                                                                 s |
                |             b                                                                                   c |
                |             e                                                                                   i |
                |             l                                                                                   e |
                |             l                                                                                   n |
                |             e                    g                                                              c |
                |      a      s      e             o                                                              e |
                |      d      _      d             v                                         r                    _ |
                |      v      l      i      f      e    

In [18]:
#The confusion matrix for the decision tree classifier
test = decision_tree_classifier.classify_many(f for (f, c) in test_set)
cm = nltk.ConfusionMatrix(gold, test)
print(cm.pretty_format(sort_by_count=False, show_percents=True, truncate=15))

                |                                                                                                 s |
                |             b                                                                                   c |
                |             e                                                                                   i |
                |             l                                                                                   e |
                |             l                                                                                   n |
                |             e                    g                                                              c |
                |      a      s      e             o                                                              e |
                |      d      _      d             v                                         r                    _ |
                |      v      l      i      f      e    

## Final discussion

Neither of the classifiers is not very accurate. The Naive Bayes classifier's performance varies a lot from run to run. Its performance varies from about 30% to around 60% which is not very good. The decision tree performs worse than the Naive Bayes classifier on every run.

Compared to the majority baseline the Naive Bayes classifier performs always better than 16% so one could say that this classifier could be somewhat useful. The decision tree performs sometimes worse than what the majority baseline is, so it is not really a classifier that should be used at all.
